In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [2]:
my $practice_input = qq/2333133121414131402/;

2333133121414131402

In [28]:
sub render_partition_map {
    my ($input) = @_;

    join '', @{
        map_nd_indexed sub {
            my ($arr, $i) = @_;
            $i = $i->[0];
            my ($file, $free) = split '', $arr->[$i];
            return $i x $file . '.' x $free;
            # return $v;
        },
        [ unpack('(A2)*', $input) ]
    };
}

sub checksum_partition_map { sum @{ map_nd_indexed sub { $_[0][$_[1][0]] * $_[1][0] }, $_[0] } }


use feature qw/ say state /;

sub process_day9_part1 {
    my ($input) = @_;
    my @shape = shape(parse_2d_map_array($input));

    # render_partition_map($input);
    # my $partition_map_rendered = join '', @{
    my $sector_map =
        map_nd_indexed sub {
            my ($arr, $i) = @_;
            $i = $i->[0];
            my ($file, $free) = split '', $arr->[$i];
            $free ||= 0;
            return ( map { { id => $i, free => 0 } } 1 .. $file ), (map { { id => -1, free => 1 } } 1 .. $free);
        },
        [ unpack('(A2)*', $input) ];

    my $count_non_free = grep { not $_->{free} } @$sector_map;

    my ($mid_point) =
        grep $_->{next_taken} == $_->{previous_free},
        @{
            map_nd_indexed sub {
                my ($arr, $i) = @_;
                $i = $i->[0];
                state $previous_free //= 0;
                state $previous_taken //= 0;
                $arr->[$i]{previous_free} = $previous_free;
                $arr->[$i]{next_taken} = $count_non_free - $previous_taken;
                $arr->[$i]{index} = $i;
                $previous_free++ if $arr->[$i]{free};
                $previous_taken++ unless $arr->[$i]{free};
                $arr->[$i];
            },
            $sector_map
        };
    say "debug: midpoint:", Dumper $mid_point;

    my @inverse_list = reverse grep { not $_->{free} } @{$sector_map}[$mid_point->{index} .. $#$sector_map];

    my $transformed_sector_map =
    # join '',
    # @{
        map_nd selector('id'),
        map_nd sub {
            state $free_index //= 0;
            $_->{free} ? $inverse_list[$free_index++] : $_;
        },
        [ @{$sector_map}[0 .. $mid_point->{index} - 1] ];

    my $rendered_transformed_sector_map = join '', @$transformed_sector_map;

    # say "debug: rendered_transformed_sector_map: $rendered_transformed_sector_map";
    checksum_partition_map($transformed_sector_map);
    # join '', @$transformed_sector_map;
}

say Dumper process_day9_part1($practice_input);

debug: midpoint:$VAR1 = {
          'free' => 0,
          'id' => 6,
          'index' => 28,
          'next_taken' => 12,
          'previous_free' => 12
        };

$VAR1 = 1928;



1

Warning: Subroutine render_partition_map redefined at reply input line 1.

Subroutine checksum_partition_map redefined at reply input line 16.

Subroutine process_day9_part1 redefined at reply input line 21.


In [29]:
my $input = get_challenge('2024/day/9/input');
my $res = process_day9_part1($input);
# 6446899523367

debug: midpoint:$VAR1 = {
          'free' => 1,
          'id' => -1,
          'index' => 50338,
          'next_taken' => 23808,
          'previous_free' => 23808
        };



6446899523367

In [76]:
say post_answer('2024/day/9/answer', 1, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 9 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're 

1

In [82]:
use feature qw/ say state /;


$graphql::graphql_methods{group_by} = sub {
        my ($self, $params) = @_;
        my $key = $params->{key};
        my $arr = $self->{arr};

        state $cached //= {};

        $cached->{"$arr/$key"} = {
            group_by selector($key),
            flatten_nd
            map_nd_indexed graphql_query("{ value }"),
            $self->{arr}
        } unless exists $cached->{"$arr/$key"};
        return $cached->{"$arr/$key"};
    };

sub render_sector_map {
    my @rendered_sector_map;
    foreach my $file (grep !$_->{free}, @_) {
        @rendered_sector_map[map $file->{index} + $_, 0 .. $file->{length}-1] = ($file->{id}) x $file->{length};
    }
    return join '', map defined ? $_ : '.', @rendered_sector_map;
}

sub naturalize_sector_map {
    my @rendered_sector_map;
    foreach my $file (grep !$_->{free}, @_) {
        @rendered_sector_map[map $file->{index} + $_, 0 .. $file->{length}-1] = ($file->{id}) x $file->{length};
    }
    return @rendered_sector_map;
}

sub process_day9_part2 {
    my ($input) = @_;
    my @shape = shape(parse_2d_map_array($input));

    # render_partition_map($input);
    # my $partition_map_rendered = join '', @{
    my @sector_map = @{
        map_nd_indexed sub {
            my ($arr, $i) = @_;
            $i = $i->[0];
            state $offset //= 0;
            my ($file, $free) = split '', $arr->[$i];
            $free ||= 0;
            my @blocks;
            push @blocks, { length => $file, index => $offset, free => 0, id => $i } if $file > 0;
            push @blocks, { length => $free, index => $offset + $file, free => 1 } if $free > 0;

            $offset += $file + $free;
            return @blocks;
        },
        [ unpack('(A2)*', $input) ]
    };
    # my @free_blocks = grep $_->{free}, @$sector_map;
    # my @file_blocks = grep !$_->{free}, @$sector_map;

    # my %free_chains =
    #     group_by selector('length'),
    #     # map_nd_indexed graphql_query('{ group_by(key:"value.length") }'),
    #     @free_blocks;

    # foreach my $file (reverse @file_blocks) {
    #     if $free_chains{
    # }

    # say "debug: rendered_sector_string: ", render_sector_map(@sector_map);

    my @files_list = grep !$_->{free}, @sector_map;
    foreach my $file (reverse @files_list) {
        
        # my $file = (grep !$_->{free}, @sector_map)[-1];
    
        my $space = (
            sort { $a->{index} <=> $b->{index} }
            grep $file->{index} > $_->{index},
            grep $file->{length} <= $_->{length},
            grep $_->{free},
            @sector_map)[0];
    
        if (defined $space) {
            push @sector_map, { length => $file->{length}, index => $file->{index}, free => 1 };
            $file->{index} = $space->{index};
            $space->{index} += $file->{length};
            $space->{length} -= $file->{length};
            if ($space->{length} == 0) {
                @sector_map = grep $_ != $space, @sector_map;
            }
            # say "debug: rendered_sector_string: ", render_sector_map(@sector_map);
        }

    }
    say "debug: final rendered_sector_string: ", render_sector_map(@sector_map);
    
    # naturalize_sector_map(@sector_map);
    checksum_partition_map([ map $_ ? $_ : 0, naturalize_sector_map(@sector_map) ]);
}

# say Dumper process_day9_part2("2333133121414131402");
say Dumper process_day9_part2($practice_input);


debug: final rendered_sector_string: 00992111777.44.333....5555.6666.....8888
$VAR1 = 2858;



1

Warning: Subroutine render_sector_map redefined at reply input line 20.

Subroutine naturalize_sector_map redefined at reply input line 28.

Subroutine process_day9_part2 redefined at reply input line 36.


In [83]:
my $input = get_challenge('2024/day/9/input');
my $res = process_day9_part2($input);
# 6446899523367

debug: final rendered_sector_string: ....4058405840584058405840584058.67756775677567756775677567756775677540604060677467746774677467746774677467746774.6299629962996299.406240624062406240624062406240626295629562956295.406340634063629162916291629140644064406440644065406540654065406540654065635363536353635363536353.6343634363436343634363434067406740674067406740674067406740676277627762776277406840684068406840684068406862606260626062604069406962586258625862584070407040704070407040704070407040706335633563356335633563354071407140714071407140714071407165736573657365736573657365736573407240724072407240724072407240724072.407340734073407340734073407440744074407440744074407440744074407540754075407540754075407540764076407640764076.407740774077407740774077407740774077632763276327632763276327.6289628962896289628940794079407940794079407940796384638463846384638463846384408040804080408060096009600940814081408140816767676767676767676767676767676767674082408240824082408240825997599759974083408340834083408

6478232739671

In [86]:
# 6478232739671
say post_answer('2024/day/9/answer', 2, $res);

1